In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import nibabel as nib

from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan

def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    return volume

### Touch level 1,4 train/val data set

In [14]:
touch_level_1_paths = [
    os.path.join(os.getcwd(), "MRI_nii/touch/level_1", x)
    for x in os.listdir("MRI_nii/touch/level_1")
]
touch_level_4_paths = [
    os.path.join(os.getcwd(), "MRI_nii/touch/level_4", x)
    for x in os.listdir("MRI_nii/touch/level_4")
]
print("Train set: MRI scans with level1 brain tissue: " + str(len(touch_level_1_paths)))
print("Train set: MRI scans with level4 brain tissue: " + str(len(touch_level_4_paths)))

Train set: MRI scans with level1 brain tissue: 579
Train set: MRI scans with level4 brain tissue: 572


In [17]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
touch_1_scans = np.array([process_scan(path) for path in touch_level_1_paths])
touch_4_scans = np.array([process_scan(path) for path in touch_level_4_paths])

touch_1_labels = np.array([1 for _ in range(len(touch_1_scans))])
touch_4_labels = np.array([0 for _ in range(len(touch_4_scans))])

# Split data into training and validation splits
x_train = np.concatenate((touch_1_scans[:round(len(touch_level_1_paths)*0.7)], touch_4_scans[:round(len(touch_level_1_paths)*0.7)]), axis=0)
y_train = np.concatenate((touch_1_labels[:round(len(touch_level_1_paths)*0.7)], touch_4_labels[:round(len(touch_level_1_paths)*0.7)]), axis=0)
x_val = np.concatenate((touch_1_scans[round(len(touch_level_1_paths)*0.7):], touch_4_scans[round(len(touch_level_1_paths)*0.7):]), axis=0)
y_val = np.concatenate((touch_1_labels[round(len(touch_level_1_paths)*0.7):], touch_4_labels[round(len(touch_level_1_paths)*0.7):]), axis=0)

print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

Number of samples in train and validation are 810 and 341.
